In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import itertools
import random
from matplotlib import pyplot as plt
from pair_trading_foundations.data_generation import ExecutePairTrading, generate_training_data
random.seed(23)
import cProfile
import pstats
import pickle
import plotly.express as px
from time import time
import warnings
warnings.filterwarnings('ignore')

def chunker(seq, size):
    # split a list into chunks
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

/tmp/ipykernel_2703/4155852776.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# data = pd.read_csv('Data/sp500_full_20181231_to_20231229.csv')
data = pd.read_csv('Data/sp500_full_19991201_to_20231231.csv')
value_count_tb = data[['Ticker']].groupby('Ticker').size().reset_index()
value_count_tb.columns = ['Ticker', 'Count']
stock_to_keep = value_count_tb['Ticker'][value_count_tb.Count==value_count_tb.Count.max()]
data = data[data.Ticker.isin(stock_to_keep)]

# Generate for all pairs

In [4]:
tickers = list(set(data.Ticker.values))
combinations = list(itertools.combinations(tickers, 2))
len(combinations)
batches = list(chunker(combinations, 1000))

In [5]:
len(data)

2169122

In [93]:
from importlib import reload
import pair_trading_foundations.data_generation
reload(pair_trading_foundations.data_generation)
from pair_trading_foundations.data_generation import ExecutePairTrading, generate_training_data

i = 0
for batch in batches:
    start_ts=time()
    print(f'Getting {i+1}th out of {len(batches)} batches')
    max_pairs = 1
    features_tb, labels_tb = generate_training_data(
        data=data,
        moving_average=50,
        training_len=300,
        test_len=50,
        entry_signal=5,
        exit_signal=3,
        calculate_label=True,
        verbose=False,
        max_combinations=max_pairs,
        combinations=batch
    )
    combined = pd.merge(features_tb, labels_tb, how='left', on=['Date', 'Ticker_P1','Ticker_P2']).reset_index(drop=True)
    combined = combined[combined.pnls.notnull()].reset_index(drop=True)
    combined.to_csv(f'Data/Training/pair_features_{i+1}_pairs{max_pairs}_300_120.csv', index=False)
    end_ts = time()
    print(f"Took {end_ts - start_ts} seconds")
    i+=1
    break
    
len(combined)
combined

Getting 1th out of 64 batches
1000 stock pairs detected
Took 0.4925498962402344 to initilize. Entering ticker pair loop
Max combination = 1
Took 4.797440052032471 to finish
Took 4.883245944976807 seconds


,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls
0,WELL,2001-02-08 00:00:00,18.850000,18.709999,61000.0,5.180556,JBHT,4.796875,4.656250,348400.0,3.770645,1.409912,1.296872,0.508011,1.228968,0.342294,0.074342
1,WELL,2001-02-09 00:00:00,18.900000,18.620001,78500.0,5.128313,JBHT,4.781250,4.656250,1077600.0,3.859216,1.269097,1.297921,0.507626,1.217006,0.329757,0.065709
2,WELL,2001-02-12 00:00:00,18.910000,18.639999,76700.0,5.142061,JBHT,4.765625,4.515625,561600.0,3.859216,1.282845,1.299126,0.507158,1.204147,0.313696,0.064923
3,WELL,2001-02-13 00:00:00,18.980000,18.650000,58600.0,5.197054,JBHT,4.812500,4.722656,1248400.0,3.868706,1.328349,1.300767,0.506450,1.193864,0.300493,0.053365
4,WELL,2001-02-14 00:00:00,19.000000,18.870001,75600.0,5.221804,JBHT,4.812500,4.656250,446400.0,3.783297,1.438506,1.302190,0.506235,1.186129,0.288654,0.075193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5704,WELL,2023-10-12 00:00:00,85.410004,84.309998,2703400.0,83.850723,JBHT,195.089996,190.860001,578800.0,192.030792,108.180069,106.504981,7.981195,109.405075,5.968722,0.000000
5705,WELL,2023-10-13 00:00:00,85.680000,84.589996,2727100.0,83.998695,JBHT,195.649994,191.100006,849200.0,190.567261,106.568565,106.520203,7.976931,109.031157,5.523957,0.000000
5706,WELL,2023-10-16 00:00:00,85.980003,84.529999,2356400.0,84.511658,JBHT,200.050003,193.229996,1028800.0,195.913605,111.401947,106.542599,7.981187,108.760936,5.040504,0.000000
5707,WELL,2023-10-17 00:00:00,86.970001,85.070000,2182700.0,85.389626,JBHT,200.199997,195.550003,1352400.0,195.147003,109.757378,106.551007,7.983248,108.492682,4.595249,0.000000


In [90]:
labels_tb
labels_tb = labels_tb.dropna()
labels_tb.pnls.describe()

count    114180.000000
mean          0.000982
std           0.015997
min          -0.258471
25%           0.000000
50%           0.000000
75%           0.000000
max           0.457658
Name: pnls, dtype: float64

In [91]:
data.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,1999-12-01 00:00:00,47.34375,47.62500,46.78125,47.21875,24.006454,2062600.0,MMM,Industrials,Industrial Conglomerates
1,1999-12-02 00:00:00,47.50000,49.21875,47.46875,48.59375,24.705523,2617400.0,MMM,Industrials,Industrial Conglomerates
2,1999-12-03 00:00:00,48.75000,50.40625,48.75000,49.65625,25.245691,3060600.0,MMM,Industrials,Industrial Conglomerates
3,1999-12-06 00:00:00,49.25000,49.53125,48.68750,48.90625,24.864393,1282200.0,MMM,Industrials,Industrial Conglomerates
4,1999-12-07 00:00:00,48.75000,48.75000,46.50000,46.50000,23.641027,2190400.0,MMM,Industrials,Industrial Conglomerates


# Generate data for sampled pairs

In [ ]:
sampled_tickers = random.sample(list(stock_to_keep.values), 50)
# data_tech = data[data['GICS Sector'].isin(['Information Technology'])]
data_sampled = data[data['Ticker'].isin(sampled_tickers)]

In [ ]:
features_tb, labels_tb, pnl_metadata_tb = generate_training_data(
        data=data_sampled,
        training_len=300,
        test_len=20,
        calculate_label=True,
        verbose=False
    )

# Write data out

In [ ]:
spy_df = pd.read_csv('Data/Training/1999-12-01-2023-12-31_SPY.csv')
spy_df = spy_df[['Date','Adj Close']]
spy_df.columns = ['Date','SPY_Close']

look_forward_d = 20
# Define a variable to calculate the return if we just buy SPY and sell in the next 60 days
spy_return = []
for i in range(spy_df.shape[0]):
    if (i + look_forward_d) < spy_df.shape[0]:
        spy_return.append(
            100*(spy_df.loc[i+look_forward_d]['SPY_Close'] - spy_df.loc[i]['SPY_Close'])/spy_df.loc[i]['SPY_Close']
        )
    else:
        spy_return.append(
            np.nan
        )
spy_df['SPY_return'] = spy_return

In [ ]:
combined = pd.merge(features_tb, labels_tb, how='left', on=['Date', 'Ticker_P1','Ticker_P2']).reset_index(drop=True)
combined = pd.merge(combined, spy_df[['Date','SPY_return']], how='left', on='Date')
combined['pnls'] = combined.pnls * 100
combined['successful_pair_trading'] = (combined.both_legs_profited) & (combined.pnls > combined.SPY_return)
combined.both_legs_profited = combined.both_legs_profited.astype(int, errors='ignore')

In [ ]:
combined.to_csv(f'Data/Training/pair_features_updated_300_20.csv', index=False)

In [ ]:
# combined = pd.merge(combined, pnl_metadata_tb[['Date', 'Ticker_P1','Ticker_P2', 'trade_executions']], how='left', on=['Date', 'Ticker_P1','Ticker_P2'])

In [ ]:
combined.columns

In [ ]:
1539825/1225

In [ ]:
with open('Data/spotcheckout_output.pkl','wb') as file:
    pickle.dump(combined, file)